In [266]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import clashroyale
from tqdm import tqdm_notebook as tqdm
import time
import random
import json
import sys   

from datetime import datetime
from dateutil import parser

In [26]:
# My Clash Royale Developer Key
# Follow https://github.com/cgrok/clashroyale#documentation to get your developer keys
import configparser
config = configparser.ConfigParser()
config.read("config.ini")
SECRETDEVKEY = config.get("Key","SECRETDEVKEY")

In [243]:
# start session
client = clashroyale.RoyaleAPI(SECRETDEVKEY)
# get the top player to start
def get_top_players():
    players = client.get_top_players() 
    return players

def get_player_battles(tag):
    battles = client.get_player_battles(tag)
    return battles

#client.close()
time_start = time.time()
players = get_top_players()
now = time.time()
print(len(players),type(players))
print("It took ", now-time_start, " to run one \"get_player_battle\" method")

200 <class 'list'>
It took  0.8217549324035645  to run one "get_player_battle" method


In [244]:
# get random player tag
random.shuffle(players)
tag = players[0].raw_data["tag"]
tag

'8VY02QL0'

In [245]:
battles = get_player_battles(tag)
battles[0].raw_data

{'arena': {'arena': 'League 5',
  'arenaID': 17,
  'name': 'Master II',
  'trophyLimit': 5200},
 'challengeType': None,
 'deckType': 'slotDeck',
 'mode': {'cardLevels': 'Ladder',
  'deck': 'Collection',
  'name': 'Ladder',
  'overtimeSeconds': 60,
  'players': 'PvP',
  'sameDeck': False},
 'opponent': [{'clan': {'badge': {'category': '01_Symbol',
     'id': 16000026,
     'image': 'https://royaleapi.github.io/cr-api-assets/badges/Skull_03.png',
     'name': 'Skull_03'},
    'name': 'BulgarianGang™',
    'tag': '8QQUCUQY'},
   'crownsEarned': 0,
   'deck': [{'arena': 4,
     'description': 'A heavily armored, slow melee fighter. Swings from the hip, but packs a huge punch.',
     'elixir': 7,
     'icon': 'https://api-assets.clashroyale.com/cards/300/MlArURKhn_zWAZY-Xj1qIRKLVKquarG25BXDjUQajNs.png',
     'id': 26000004,
     'key': 'pekka',
     'level': 8,
     'maxLevel': 8,
     'name': 'P.E.K.K.A',
     'rarity': 'Epic',
     'requiredForUpgrade': 'Maxed',
     'type': 'Troop'},
   

In [246]:
# get the next tag to have get_player_battles function can performed recursively
def next_tag(battles):
    idx = random.randint(0,len(battles)-1)
    #print (len(battles),idx)
    tag = battles[idx].raw_data["opponent"][0]["tag"]
    return tag
print(next_tag(battles))

QY82RY9P


In [324]:
#battles_df = pd.DataFrame(columns = battles[0].raw_data.keys())
#battles_list = []
battle_df_size_tracker =0
battle_df_size = 50000
             
with open('battles_temp.json', 'r+') as f:
    battle_df_size_tracker = sum(1 for row in f)
    print(battle_df_size_tracker)
    with tqdm(total=battle_df_size, initial=battle_df_size_tracker) as pbar:
        while True:
            while True:
                try:
                    temp_battles = get_player_battles(tag)
                except Exception as e:
                    e = sys.exc_info()
                    print (e)
                if not temp_battles:
                    tag = next_tag(battles)
                else:
                    battles = temp_battles
                    break
            if not isinstance(battles,list):
                battles = list(battles)
            #print (len(battles))
            dt = datetime.utcfromtimestamp((battles[-1].raw_data["utcTime"]))
            if dt.month == datetime.now().month-1:
                #random.shuffle(battles)
                #battles = battles[0:10] if len(battles) > 10 else battles
                #battles_list.extend(battles)
                for battle in battles:
                    json.dump(battle.raw_data, f) 
                #battles_df = pd.concat([battles_df, pd.DataFrame(battle.raw_data for battle in battles)],sort=False)
                pbar.update(len(battles))
                pbar.total
                #print(battles_list)
                #reader = 
                row_count = sum(1 for row in f)
                if row_count > battle_df_size:
                    break
            tag = next_tag(battles)

0


HBox(children=(IntProgress(value=0, max=50000), HTML(value='')))

(<class 'clashroyale.errors.NotResponding'>, NotResponding('API request timed out, please be patient.',), <traceback object at 0x000002438BC77188>)
(<class 'clashroyale.errors.NotResponding'>, NotResponding('API request timed out, please be patient.',), <traceback object at 0x000002438BC77988>)
(<class 'clashroyale.errors.NotResponding'>, NotResponding('API request timed out, please be patient.',), <traceback object at 0x000002438BC77E48>)
(<class 'clashroyale.errors.NotResponding'>, NotResponding('API request timed out, please be patient.',), <traceback object at 0x000002438C113148>)
(<class 'clashroyale.errors.NotResponding'>, NotResponding('API request timed out, please be patient.',), <traceback object at 0x000002438BC68DC8>)
(<class 'clashroyale.errors.NotResponding'>, NotResponding('API request timed out, please be patient.',), <traceback object at 0x000002438BC68448>)
(<class 'clashroyale.errors.NotResponding'>, NotResponding('API request timed out, please be patient.',), <trace

KeyboardInterrupt: 

In [393]:
data = []

with open("battles_temp.json", "r") as f:
    with tqdm(total=650, initial=0) as pbar:
        for item in f:
            while item:
                try:
                    json.loads(item)
                except Exception as e:
                    e = sys.exc_info()[1]
                    num_index_start = (str(e).rfind("(char")+len("(char"))
                    file_start_idx = int(str(e)[num_index_start:-1])
                    if file_start_idx == 1:
                        break
                    data.append(json.loads(item[:file_start_idx]))
                    item=item[file_start_idx:]
                    pbar.update(1)

HBox(children=(IntProgress(value=0, max=650), HTML(value='')))

KeyboardInterrupt: 

In [403]:
dt = pd.DataFrame(data).utcTime.apply(lambda x: datetime.utcfromtimestamp(x))
# check month distribution of the dataset
dt.apply(lambda x:x.month).value_counts()
dt.name = "month"

In [405]:
df = pd.DataFrame(data).join(dt.apply(lambda x:x.month))

In [413]:
df = df[df["type"]=="PvP"]
df.month.value_counts()

9     159
10    154
Name: month, dtype: int64

In [458]:
from collections import Counter
c = Counter()
df.team.apply(lambda x:x[0]["deck"])

2      [{'name': 'Inferno Tower', 'level': 7, 'maxLev...
3      [{'name': 'Inferno Tower', 'level': 7, 'maxLev...
9      [{'name': 'Inferno Tower', 'level': 7, 'maxLev...
19     [{'name': 'Baby Dragon', 'level': 4, 'maxLevel...
22     [{'name': 'Baby Dragon', 'level': 4, 'maxLevel...
23     [{'name': 'Baby Dragon', 'level': 4, 'maxLevel...
24     [{'name': 'Baby Dragon', 'level': 4, 'maxLevel...
25     [{'name': 'Baby Dragon', 'level': 4, 'maxLevel...
27     [{'name': 'Baby Dragon', 'level': 4, 'maxLevel...
28     [{'name': 'Baby Dragon', 'level': 4, 'maxLevel...
29     [{'name': 'Baby Dragon', 'level': 4, 'maxLevel...
31     [{'name': 'Rage', 'level': 3, 'maxLevel': 8, '...
33     [{'name': 'Spear Goblins', 'level': 8, 'maxLev...
44     [{'name': 'Baby Dragon', 'level': 4, 'maxLevel...
47     [{'name': 'Baby Dragon', 'level': 4, 'maxLevel...
48     [{'name': 'Baby Dragon', 'level': 4, 'maxLevel...
49     [{'name': 'Baby Dragon', 'level': 4, 'maxLevel...
50     [{'name': 'Baby Dragon',